# Data loading

MSGPACK --> Parquet.

In [ ]:
from ship_routing.app.routing import RoutingResult, RoutingLog
from load_tuning_results import (
    load_results_raw,
    load_result_for_key,
    load_results,
    get_journey_params_df,
    get_hyper_params_df,
    get_runtime_df,
    get_elite_df,
)

In [2]:
from pathlib import Path

import pandas as pd
import geopandas as gpd

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
data_files = sorted(Path("results/").glob("results_1*.msgpack"))
results = load_results(data_files)
len(results)

records:   0%|          | 0/168471 [00:00<?, ?it/s]

In [ ]:
# Load individual DataFrames
df_hyper = get_hyper_params_df(results)
df_journey = get_journey_params_df(results)
df_runtime = get_runtime_df(results)
df_elite = get_elite_df(results)

# Merge: params (1-to-1) → runtime (1-to-1) → elite (1-to-many with left join)
df_merged = df_hyper.merge(
    df_journey, left_index=True, right_index=True, how="inner"
).merge(
    df_runtime, left_index=True, right_index=True, how="inner"
).merge(
    df_elite, left_index=True, right_index=True, how="left"
)

df_merged

elite:   0%|          | 0/3983 [00:00<?, ?it/s]

,hyper_population_size,hyper_random_seed,hyper_selection_acceptance_rate_warmup,hyper_mutation_width_fraction_warmup,hyper_mutation_displacement_fraction_warmup,hyper_generations,hyper_offspring_size,hyper_crossover_rounds,hyper_selection_quantile,hyper_selection_acceptance_rate,...,journey_speed_knots,journey_time_resolution_hours,runtime,runtime_seconds,n_elite,elite_length_meters,elite_length_relative,elite_cost_absolute,elite_cost_relative,geometry
filename,,,,,,,,,,,,,,,,,,,,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,32,202838879,0.3,0.5,0.10,4,32,0,0.10,0.00,...,10.0,6.0,0 days 00:01:16,76.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099..."
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,32,240864382,0.3,0.9,0.10,4,32,1,0.10,0.25,...,10.0,6.0,0 days 00:01:37,97.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099..."
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10.0,6.0,0 days 00:04:02,242.0,0,6.102691e+06,1.001534,9.154279e+12,0.948557,"LINESTRING (-11 50.00000000000001, -12.5311121..."
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10.0,6.0,0 days 00:04:02,242.0,1,6.096694e+06,1.000550,9.457689e+12,0.979995,"LINESTRING (-11 50, -12.527629087397221 50.105..."
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,32,3459840799,0.3,0.5,0.25,4,32,2,0.25,0.25,...,10.0,6.0,0 days 00:03:33,213.0,0,6.237164e+06,1.023603,1.018871e+13,0.000000,"LINESTRING (-80.5 29.99999999999999, -79.65863..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
result:2025-12-06T17-39-09.233_f3289d51-7e95-4da3-a9de-54f5a06c8007,128,3285732148,0.3,0.9,0.10,4,128,2,0.10,0.00,...,10.0,6.0,0 days 00:05:55,355.0,1,6.095589e+06,1.000369,6.512015e+12,0.981262,"LINESTRING (-11 50.00000000000001, -12.5118055..."
result:2025-12-06T16-44-21.729_b8f12204-b323-4017-9641-ea90d16d8bb9,32,3136936006,0.3,0.5,0.10,1,32,2,0.25,0.00,...,10.0,6.0,0 days 00:01:42,102.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-80.5 30, -79.64621998055947 30.67..."
result:2025-12-06T17-50-07.470_8fe43914-cffb-4c57-b186-2d112f696d9f,32,1139659395,0.3,0.5,0.25,1,32,2,0.10,0.25,...,10.0,6.0,0 days 00:01:18,78.0,0,6.096115e+06,1.000455,6.551044e+12,0.987143,"LINESTRING (-11 50.00000000000001, -12.5032048..."


In [ ]:
gpd.GeoDataFrame(df_merged).to_parquet("results/results_prelim.geoparquet")